In [20]:
import yfinance as yf
import pandas as pd
from datetime import datetime

ticker = yf.Ticker("AAPL")


In [21]:
def get_major_news(ticker):
    news = ticker.news
    data = []
    for item in news:
        title = item.get('title')
        link = item.get('link')
        source = item.get('publisher')
        # Convertir le timestamp Unix en format lisible
        publish_time = datetime.fromtimestamp(item.get('providerPublishTime')).strftime('%Y-%m-%d %H:%M:%S')
        
        # Ajouter les informations dans un dictionnaire
        data.append({
            'Title': title,
            'Link': link,
            'Source': source,
            'Publish Time': publish_time
        })
    
    # Convertir la liste en DataFrame
    df = pd.DataFrame(data)
    return df


In [29]:
print(get_major_news(ticker)["Link"][0])

https://finance.yahoo.com/news/apple-intelligence-products-could-apple-161448615.html


In [30]:
import requests
from bs4 import BeautifulSoup
import json

# Fonction pour extraire le texte principal de l'article
def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraction des paragraphes de l'article
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text() for para in paragraphs])
    return article_text

In [31]:
extract_article_text(get_major_news(ticker)["Link"][0])

"We are experiencing some temporary issues. The market data on this page is currently delayed. Please bear with us as we address this and restore your personalized lists. As smartphone companies fight to be the first to launch the latest generative AI features in their iPhones, Apple is taking its time. The company is facing weak iPhone sales and seems to be behind in the AI race. But all that could change once the first Apple Intelligence device hits the market. Apple Inc. (AAPL) is a multinational technology powerhouse that designs, develops, and markets electronic devices, software, and services. The core product offerings include iPhone, iPad, MacBook, Apple Watch, and Apple TV. With a firm emphasis on visual aesthetics, user interaction, and smooth unification of hardware and software, the company has set itself apart from the competition. Apple has built a strong ecosystem that cultivates brand loyalty through integrated product portfolios, including iCloud and App Store. The lea

In [32]:
import logging
import boto3
from botocore.exceptions import ClientError

In [40]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [41]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
session_token = os.getenv("AWS_SESSION_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")


In [42]:

bedrock_client = boto3.client('bedrock-runtime', region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  )


In [60]:
# Fonction pour analyser le sujet principal
def analyze_subject(text):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 50,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Quel est le sujet principal de cet article : \"{text}\""}
                ]
            })
        )
        
        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        subject_analysis = content[0]["text"] if content else "Pas de sujet détecté"
        return subject_analysis
    
    except Exception as e:
        print("Erreur d'analyse du sujet :", e)
        return None



In [71]:
# Fonction pour l'analyse de sentiment
def analyze_sentiment(text):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 50,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Analyse le sentiment de ce texte : \"{text}\""}
                ]
            })
        )

        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        sentiment_analysis = content[0]["text"] if content else "Pas de sentiment détecté"
        return sentiment_analysis
    
    except Exception as e:
        print("Erreur d'analyse de sentiment :", e)
        return None



In [ ]:

url = "https://finance.yahoo.com/news/apple-intelligence-products-could-apple-161448615.html"
article_text = extract_article_text(url)



In [ ]:

sujet_principal = analyze_subject(article_text)
print("Sujet Principal de l'Article :", sujet_principal)


Sujet Principal de l'Article : D'après le contenu de l'article, le sujet principal semble être les perspectives d'Apple dans la course à l'intelligence artificielle, notamment avec le lancement à venir de son appareil "Apple Intelligence" et


In [74]:

sentiment_principal = analyze_sentiment(article_text)

print("Sentiment Principal de l'Article :", sentiment_principal)

Sentiment Principal de l'Article : D'après l'analyse du sentiment de ce texte, le ton général semble être plutôt neutre et informatif, avec quelques éléments positifs et négatifs. Voici un résumé des princip
